# Hi Jon/Sonam, lets start here!


#### Only if you were redirected here from the New CUP, you must be in the year of decarb with New CUP results and now you can rerun this file for final results


## First, pick a year - no decarb (2025) or decarb (2045)? Type it in OP_building_loads.py and then hit "Run All" above in this file


In [1]:
import os
import pandas as pd
import shutil
import re
import pprint
import math
import numpy as np

In [2]:
# loading results from OP_building_loads file
from districtsystem.OP_building_loads import (
    allBldgLoads_output,
    allBldgElecUse_output,
    allBldgGasUse_output,
    current_district_therm_loads,
    COP_wetbulb_reg_output,
    current_District_Elec_Use,
    current_District_Gas_Use,
    current_Bldg_Therm_Op_Loads,
    Current_Bldg_Equip_Gas_Use,
    Current_Bldg_Equip_Elec_Use,
    new_Bldg_Therm_Op_Loads,
    COP_newBldgEquip_reg_output,
    new_bldg_equip_elec_Use,
    newCUp_reg_df,
    bldg_total_elec_use,
    bldg_total_gas_use,
    rates_df,
    bldg_total_elec_grouped_df,
    new_CUP_Therm_Loads,
    key_outputs_OG,
    new_CUP_equip_elec_Use,
    elec_billing_df,
)

/Users/samueld/PycharmProjects/district-system/districtsystem/OP_building_loads.py:1515: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3_E_CAAN[col] = 0  # Initialize columns with 0
/Users/samueld/PycharmProjects/district-system/districtsystem/OP_building_loads.py:1515: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3_E_CAAN[col] = 0  # Initialize columns with 0
/Users/samueld/PycharmProjects/district-system/districtsystem/OP_building_loads.py:1515: SettingWithCopyWarning: 
A value is trying to be set on

In [3]:
# simulation thermal Use
allBldgLoads_output.iloc[:, :]

,Timestamp,Building ID CAAN,Cooling Load (kbtu),Heating Load (kbtu),Hot Water Load (kbtu),Cooking Load (kbtu),Laundry Load (kbtu)
0,2010-01-01 00:00:00,81865,0.0,75.780583,0.043743,0.0,0.000000
1,2010-01-01 01:00:00,81865,0.0,79.728970,0.048317,0.0,0.016199
2,2010-01-01 02:00:00,81865,0.0,81.036844,0.037055,0.0,0.018750
3,2010-01-01 03:00:00,81865,0.0,80.610301,0.036689,0.0,0.000000
4,2010-01-01 04:00:00,81865,0.0,72.891201,0.020392,0.0,0.000000
...,...,...,...,...,...,...,...
2934595,2010-12-31 19:00:00,8832,0.0,3.377490,0.033585,0.0,0.001875
2934596,2010-12-31 20:00:00,8832,0.0,3.110924,0.034892,0.0,0.000000
2934597,2010-12-31 21:00:00,8832,0.0,3.194271,0.035576,0.0,0.000286
2934598,2010-12-31 22:00:00,8832,0.0,4.936392,0.035084,0.0,0.000000


In [4]:
# total number of buildngs being processed
print(allBldgLoads_output.shape[0] / 8760)

335.0


In [5]:
print(allBldgLoads_output.shape)

(2934600, 7)


In [6]:
# simulation elec use
allBldgElecUse_output.head()

,Timestamp,Building ID CAAN,Cooling (kWh),Heating (kWh),Hot Water (kWh),Cooking (kWh),Laundry (kWh),Other Process (kWh),Plug Loads (kWh),Lighting (kWh),Fans (kWh),Pumps (kWh)
0,2010-01-01 00:00:00,81865,0.0,0.0,0.0,0.0,0.0,0.0,1.72275,0.717812,1.057596,0.007698
1,2010-01-01 01:00:00,81865,0.0,0.0,0.0,0.0,0.0,0.0,1.72275,0.717812,1.057596,0.008810
2,2010-01-01 02:00:00,81865,0.0,0.0,0.0,0.0,0.0,0.0,1.72275,0.717812,1.057596,0.009257
3,2010-01-01 03:00:00,81865,0.0,0.0,0.0,0.0,0.0,0.0,1.72275,0.717812,1.057596,0.009221
4,2010-01-01 04:00:00,81865,0.0,0.0,0.0,0.0,0.0,0.0,1.72275,0.717812,1.057596,0.007270


In [7]:
# simulation gas use
allBldgGasUse_output.head()

,Timestamp,Building ID CAAN,Heating (Therms),Hot Water (Therms),Cooking (Therms),Laundry (Therms),Other Process (Therms)
0,2010-01-01 00:00:00,81865,1.515612,0.046045,0.0,0.000000,0.0
1,2010-01-01 01:00:00,81865,1.594579,0.050860,0.0,0.020249,0.0
2,2010-01-01 02:00:00,81865,1.620737,0.039006,0.0,0.023437,0.0
3,2010-01-01 03:00:00,81865,1.612206,0.038620,0.0,0.000000,0.0
4,2010-01-01 04:00:00,81865,1.457824,0.021465,0.0,0.000000,0.0


In [8]:
# current district therm Op Loads
current_district_therm_loads.head()

,index,Current District Cooling Load (kBtu),Current District Heating Load (kBtu),Current District Hot Water Load (kBtu),Ambient Air Wet Bulb Temp (F)
0,NaN,NaN,NaN,NaN,49.501190
1,NaN,NaN,NaN,NaN,49.524185
2,NaN,NaN,NaN,NaN,49.515631
3,NaN,NaN,NaN,NaN,48.960694
4,NaN,NaN,NaN,NaN,48.592221


In [9]:
# Reg. Data - Current District
COP_wetbulb_reg_output.head()

,Current District Wet Bulb (F),Current District Cooling COP
0,24,6.0
1,25,6.0
2,26,6.0
3,27,6.0
4,28,6.0


In [10]:
current_District_Elec_Use.head()

,Current District Cooling COP_WB,Current District System Cooling Electricity Use (kWh)
0,6.0,NaN
1,6.0,NaN
2,6.0,NaN
3,6.0,NaN
4,6.0,NaN


In [11]:
current_District_Gas_Use.head()

,Current District System Heating Gas Use (therms),Current District System Hot Water Gas Use (therms)
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN


In [12]:
current_Bldg_Therm_Op_Loads.head()

,cooling CAAN,Current Building Equipment Cooling Load (kBtu),heating CAAN,Current Building Equipment Heating Load (kBtu),hot water CAAN,Current Building Equipment Hot Water Load (kBtu),cooking CAAN,Current Building Equipment Cooking Load (kBtu),laundry CAAN,Current Building Equipment Laundry Load (kBtu),Building ID CAAN,Ambient Air Wet Bulb Temp (F),Timestamp


In [13]:
Current_Bldg_Equip_Elec_Use.head()

,heating CAAN,Current Building Equipment Heating Electricity Use (kWh),hot water CAAN,Current Building Equipment Hot Water Electricity Use (kWh),cooking CAAN,Current Building Equipment Cooking Electricity Use (kWh),laundry CAAN,Current Building Equipment Laundry Electricity Use (kWh),other CAAN,Current Building Equipment Other Process Electricity Use (kWh),cooling CAAN,Current Building Equipment Cooling Electricity Use (kWh),Building ID CAAN,Ambient Air Wet Bulb Temp (F),Timestamp


In [14]:
Current_Bldg_Equip_Gas_Use.head()

,heating CAAN,Current Building Equipment Heating Gas Use (therms),hot water CAAN,Current Building Equipment Hot Water Gas Use (therms),cooking CAAN,Current Building Equipment Cooking Gas Use (therms),laundry CAAN,Current Building Equipment Laundry Gas Use (therms),other CAAN,Current Building Equipment Other Process Gas Use (therms),Building ID CAAN,Ambient Air Wet Bulb Temp (F),Timestamp


In [15]:
new_Bldg_Therm_Op_Loads.head()

,cooling CAAN,Current New Building Independent Heat Pump Cooling Load (kBtu),heating CAAN,Current New Building Independent Heat Pump Heating Load (kBtu),hot water CAAN,Current New Building Independent Heat Pump Hot Water Load (kBtu),cooking CAAN,Current New Building Independent Heat Pump Cooking Load (kBtu),laundry CAAN,Current New Building Independent Heat Pump Laundry Load (kBtu),Building ID CAAN,Ambient Air Wet Bulb Temp (F),Timestamp
0,81865,0.0,81865,75.780583,81865,0.043743,81865,0.0,81865,0.000000,81865,49.501190,2010-01-01 00:00:00
1,81865,0.0,81865,79.728970,81865,0.048317,81865,0.0,81865,0.016199,81865,49.524185,2010-01-01 01:00:00
2,81865,0.0,81865,81.036844,81865,0.037055,81865,0.0,81865,0.018750,81865,49.515631,2010-01-01 02:00:00
3,81865,0.0,81865,80.610301,81865,0.036689,81865,0.0,81865,0.000000,81865,48.960694,2010-01-01 03:00:00
4,81865,0.0,81865,72.891201,81865,0.020392,81865,0.0,81865,0.000000,81865,48.592221,2010-01-01 04:00:00


In [16]:
# from districtsystem.OP_building_loads import current_district_therm_loads, COP_wetbulb_reg_output,current_District_Elec_Use,current_District_Gas_Use,current_Bldg_Therm_Op_Loads,Current_Bldg_Equip_Gas_Use,Current_Bldg_Equip_Elec_Use,\
# new_Bldg_Therm_Op_Loads,COP_newBldgEquip_reg_output,new_bldg_equip_elec_Use,new_CUP_Therm_Loads,new_CUP_equip_elec_Use,elec_billing_df

new_Bldg_Therm_Op_Loads.shape

(1620600, 13)

In [17]:
COP_newBldgEquip_reg_output.head()

,Ambient Air Dry Bulb Temp (F),New Building Independent Heat Pump - Cooling COP (44°F CHWST),New Building Independent Heat Pump - Heating COP (170°F HWST),New Building Independent Heat Pump - Hot Water COP,New Building Cooking COP,New Building Laundry COP
0,40,NaN,1.612789,2.955639,0.8,1.0
1,41,NaN,1.640132,2.981880,0.8,1.0
2,42,NaN,1.667475,3.008120,0.8,1.0
3,43,NaN,1.694817,3.034361,0.8,1.0
4,44,NaN,1.722160,3.060602,0.8,1.0


In [18]:
new_bldg_equip_elec_Use.head()

,Timestamp,Current New Bldg Cooling COP_DB,New Building Independent Heat Pump Cooling Electricity Use,Current New Bldg Heating COP_DB,New Building Independent Heat Pump Heating Electricity Use (kWh),Current New Bldg Hot water COP_DB,New Building Independent Heat Pump Hot Water Electricity Use (kWh),Current New Bldg Cooking COP_DB,New Building Equipment Cooking Electricity Use (kWh),Current New Bldg Laundry COP_DB,New Building Equipment Laundry Electricity Use (kWh),Building ID CAAN
0,2010-01-01 00:00:00,5.142933,0.0,1.995588,37.974066,3.323008,0.013164,0.8,0.0,1.0,0.000000,81865
1,2010-01-01 01:00:00,5.029448,0.0,2.050273,38.886996,3.375489,0.014314,0.8,0.0,1.0,0.016199,81865
2,2010-01-01 02:00:00,5.142933,0.0,1.995588,40.608007,3.323008,0.011151,0.8,0.0,1.0,0.018750,81865
3,2010-01-01 03:00:00,5.142933,0.0,1.995588,40.394264,3.323008,0.011041,0.8,0.0,1.0,0.000000,81865
4,2010-01-01 04:00:00,5.142933,0.0,1.995588,36.526181,3.323008,0.006137,0.8,0.0,1.0,0.000000,81865


In [19]:
new_bldg_equip_elec_Use.shape

(1620600, 12)

In [20]:
new_CUP_Therm_Loads.head()

,cooling CAAN,Current New CUP Cooling Load (kBtu),heating CAAN,Current New CUP Heating Load (kBtu),hot water CAAN,Current New CUP Hot Water Load (kBtu),cooking CAAN,Current New CUP Cooking Load (kBtu),laundry CAAN,Current New CUP Laundry Load (kBtu),Building ID CAAN,Ambient Air Wet Bulb Temp (F),Timestamp
0,8517,0.0,8517,11.360497,8517,0.000000,8517,0.0,8517,0.0,8517,49.501190,2010-01-01 00:00:00
1,8517,0.0,8517,13.899768,8517,0.001166,8517,0.0,8517,0.0,8517,49.524185,2010-01-01 01:00:00
2,8517,0.0,8517,16.125689,8517,0.000000,8517,0.0,8517,0.0,8517,49.515631,2010-01-01 02:00:00
3,8517,0.0,8517,17.244490,8517,0.001166,8517,0.0,8517,0.0,8517,48.960694,2010-01-01 03:00:00
4,8517,0.0,8517,15.584648,8517,0.001166,8517,0.0,8517,0.0,8517,48.592221,2010-01-01 04:00:00


In [21]:
new_CUP_Therm_Loads.shape

(1314000, 13)

In [22]:
# Now 'df' contains the DataFrame loaded from the pickle file
key_outputs_OG.iloc[:, 45:65]  # Verify the DataFrame

,HP_H-GrS-HHW Entering (F),HP_H-GrS-HHW Leaving (F),HP_H-GrS-HHW Load (btu),HP_H-GrS-HHW Flow (gpm),HP_H-GrS-Water-Source Entering (F),HP_H-GrS-Water-Source Leaving (F),HP_H-GrS-Water-Source Flow (gpm),HP_H-GrS-Water-Source Load (btu),HP_H-OcS-HHW Entering (F),HP_H-OcS-HHW Leaving (F),HP_H-OcS-HHW Load (btu),HP_H-OcS-HHW Flow (gpm),HP_H-OcS-Water-Source Entering (F),HP_H-OcS-Water-Source Leaving (F),HP_H-OcS-Water-Source Flow (gpm),HP_H-OcS-Water-Source Load (btu),HP_C-GrS-CHW Entering (F),HP_C-GrS-CHW Leaving (F),HP_C-GrS-CHW Load (btu),HP_C-GrS-CHW Flow (gpm)
0,106.986043,170,0.0,0.0,50,60.0001,0.0,0.0,106.986043,170,0.0,0.0,45.9399,55.9399,0.0,0.0,54.890725,44,0.0,0.0
1,106.987195,170,10037215.06724,318.576994,50,60.0001,2007.422939,10037215.06724,106.987195,170,0.0,0.0,45.9399,55.9399,0.0,0.0,54.827249,44,0.0,0.0
2,106.989673,170,10657872.973019,338.289721,50,60.0001,2131.553279,10657872.973019,106.989673,170,0.0,0.0,45.9399,55.9399,0.0,0.0,54.812509,44,0.0,0.0
3,106.989115,170,10704895.953871,339.779258,50,60.0001,2140.957781,10704895.953871,106.989115,170,0.0,0.0,45.9399,55.9399,0.0,0.0,54.824053,44,0.0,0.0
4,106.9905,170,8624759.659249,273.760613,50,60.0001,1724.934683,8624759.659249,106.9905,170,0.0,0.0,45.9399,55.9399,0.0,0.0,54.870586,44,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,106.936625,170,20000000.0,634.282574,55,65.0001,3999.96,20000000.0,106.936625,170,1000000.0,31.714129,45.2199,55.2199,200.0,1000000.0,54.892898,44,0.0,0.0
8756,106.948661,170,16841967.319448,534.230283,55,65.0001,3368.35978,16841967.319448,106.948661,170,0.0,0.0,45.2199,55.2199,0.0,0.0,54.918414,44,0.0,0.0
8757,106.954629,170,17402580.022992,552.065272,55,65.0001,3480.4812,17402580.022992,106.954629,170,0.0,0.0,45.2199,55.2199,0.0,0.0,54.878147,44,0.0,0.0
8758,106.962543,170,6321654.888618,200.568206,55,65.0001,1264.318335,6321654.888618,106.962543,170,0.0,0.0,45.2199,55.2199,0.0,0.0,54.879954,44,0.0,0.0


In [23]:
# regression
newCUp_reg_df.head(10)

,Chiller COP,Heat Pump Simultaneous - COP,Heat Pump Heating Only Air-Source - COP,Heat Pump Heating Only Water-Source - COP,Heat Pump Cooling Only Air-Source - COP,Heat Pump Cooling Only Water-Source - COP,Electric Boiler - COP
0,10.127889,6.156809,2.982861,3.066653,6.965847,5.845988,0.98
1,10.136160,6.158791,2.966568,3.066671,6.786702,5.844632,0.98
2,10.138081,6.159354,2.984711,3.066709,6.981010,5.844433,0.98
3,10.136577,6.158977,2.984705,3.066700,6.981652,5.849269,0.98
4,10.130513,6.157627,2.984719,3.066722,6.984240,5.853193,0.98
5,10.124840,6.155964,2.984642,3.066608,6.986661,5.850921,0.98
6,10.126424,6.156164,2.982792,3.066552,6.966472,5.849283,0.98
7,10.147962,6.160987,2.971877,3.066486,6.840203,5.837761,0.98
8,10.148747,6.158232,2.960338,3.065508,6.722792,5.833934,0.98
9,10.200992,6.166551,2.930474,3.064223,6.388290,5.815785,0.98


In [24]:
new_CUP_equip_elec_Use.head(5)

,New CUP Electric Boiler Electricity Use (kWh),New CUP Chiller Electricity Use (kWh),New CUP Heat Pump in Simultaneous Electricity Use (kWh),New CUP Heat Pump in Heating Only with Air-Source Electricity Use (kWh),New CUP Heat Pump in Heating Only with Water-Source Electricity Use (kWh),New CUP Heat Pump in Cooling Only with Air-Source Electricity Use (kWh),New CUP Heat Pump in Cooling Only with Water-Source Electricity Use (kWh)
0,0.0,0.0,1386956.360625,0.0,0.0,0.0,0.0
1,0.0,0.0,18986.748289,0.0,3273000.396752,0.0,0.0
2,0.0,0.0,18652.490101,0.0,3475345.356658,0.0,0.0
3,0.0,0.0,18914.076368,0.0,3490688.502671,0.0,0.0
4,0.0,0.0,23210.664812,0.0,2812371.110509,0.0,0.0


In [25]:
bldg_total_elec_use.iloc[20000:20100, :]

,Sum,DataFrame,Building ID CAAN,Timestamp
20000,1.768363,new_bldg_equip_elec_Use,81870,2010-04-14 08:00:00
20001,1.766275,new_bldg_equip_elec_Use,81870,2010-04-14 09:00:00
20002,1.779344,new_bldg_equip_elec_Use,81870,2010-04-14 10:00:00
20003,1.783833,new_bldg_equip_elec_Use,81870,2010-04-14 11:00:00
20004,1.749193,new_bldg_equip_elec_Use,81870,2010-04-14 12:00:00
...,...,...,...,...
20095,44.428294,new_bldg_equip_elec_Use,81870,2010-04-18 07:00:00
20096,1.730010,new_bldg_equip_elec_Use,81870,2010-04-18 08:00:00
20097,1.763508,new_bldg_equip_elec_Use,81870,2010-04-18 09:00:00
20098,1.801704,new_bldg_equip_elec_Use,81870,2010-04-18 10:00:00


In [26]:
bldg_total_elec_use.shape

(1638120, 4)

In [27]:
bldg_total_gas_use.iloc[40000:40100, :]

,Sum,DataFrame,Building ID CAAN,Timestamp
40000,0.0,Simulations Gas Use,8511,2010-07-26 16:00:00
40001,0.0,Simulations Gas Use,8511,2010-07-26 17:00:00
40002,0.0,Simulations Gas Use,8511,2010-07-26 18:00:00
40003,0.0,Simulations Gas Use,8511,2010-07-26 19:00:00
40004,0.0,Simulations Gas Use,8511,2010-07-26 20:00:00
...,...,...,...,...
40095,0.0,Simulations Gas Use,8511,2010-07-30 15:00:00
40096,0.0,Simulations Gas Use,8511,2010-07-30 16:00:00
40097,0.0,Simulations Gas Use,8511,2010-07-30 17:00:00
40098,0.0,Simulations Gas Use,8511,2010-07-30 18:00:00


In [28]:
elec_billing_df.head()

,Sum,DataFrame,Building ID CAAN,Timestamp,Electric Utility Billing Group,Electricity Utility Provider
0,0.0,current_District_Elec_Use,current_District_Elec_Use,2010-01-01 00:00:00,UCOP Main,UCOP
1,0.0,current_District_Elec_Use,current_District_Elec_Use,2010-01-01 01:00:00,UCOP Main,UCOP
2,0.0,current_District_Elec_Use,current_District_Elec_Use,2010-01-01 02:00:00,UCOP Main,UCOP
3,0.0,current_District_Elec_Use,current_District_Elec_Use,2010-01-01 03:00:00,UCOP Main,UCOP
4,0.0,current_District_Elec_Use,current_District_Elec_Use,2010-01-01 04:00:00,UCOP Main,UCOP


In [29]:
bldg_total_elec_grouped_df.iloc[18000:18100, :]

,Electric Utility Billing Group,Timestamp,Electricity Utility Provider,Sum,Mapped Base Rate,Mapped Escalation Rate,Base Electricity Cost ($)
18000,700125000000,2010-01-21 00:00:00,SCE,0.286087,0.129669,0.0,0.037097
18001,700125000000,2010-01-21 01:00:00,SCE,0.411234,0.126985,0.0,0.052221
18002,700125000000,2010-01-21 02:00:00,SCE,0.520518,0.125555,0.0,0.065354
18003,700125000000,2010-01-21 03:00:00,SCE,0.698200,0.125494,0.0,0.087620
18004,700125000000,2010-01-21 04:00:00,SCE,0.861731,0.127906,0.0,0.110221
...,...,...,...,...,...,...,...
18095,700125000000,2010-01-24 23:00:00,SCE,1.698801,0.117373,0.0,0.199394
18096,700125000000,2010-01-25 00:00:00,SCE,2.042805,0.129669,0.0,0.264889
18097,700125000000,2010-01-25 01:00:00,SCE,2.517430,0.126985,0.0,0.319677
18098,700125000000,2010-01-25 02:00:00,SCE,3.165779,0.125555,0.0,0.397480


In [30]:
bldg_total_elec_grouped_df["Electric Utility Billing Group"].unique()

array([700027000000, 700121000000, 700125000000, 700126000000,
       700267000000, 700268000000, 700269000000, 700345000000,
       700346000000, 700543000000, '4926667344-8', 'UCOP Main'],
      dtype=object)

In [31]:
rates_df.head()

,index,Electricity Utility Provider,Timestamp,Rates,Escalation Rate (%)
0,0,UCOP,2010-01-01 00:00:00,0.129669,0.061622
1,1,UCOP,2010-01-01 01:00:00,0.126985,0.061622
2,2,UCOP,2010-01-01 02:00:00,0.125555,0.061622
3,3,UCOP,2010-01-01 03:00:00,0.125494,0.061622
4,4,UCOP,2010-01-01 04:00:00,0.127906,0.061622


In [32]:
# Input Process and COP data from Merged baseline- Analytics tab
buildingLoadData_df = pd.read_excel(
    "UCSB merged baselines_COP_process load.xlsx", sheet_name="Analytics - V1 5-14"
)

In [33]:
# # File path for the output Excel file
# excel_file_path = 'Calculation_map_output.xlsx'


# #Write to excel
# # Create an ExcelWriter object
# with pd.ExcelWriter(excel_file_path, engine='openpyxl') as writer:
#     # Set the active sheet (first sheet) after creating the writer object
#     writer.book.active = 0


# Export each DataFrame to a different tab in the Excel file
# allBldgLoads_output.to_excel(writer, sheet_name='Simulations Thermal Use', index=False)
# allBldgElecUse_output.to_excel(writer, sheet_name='Simulations Electricity Use', index=False)
# allBldgGasUse_output.to_excel(writer, sheet_name='Simulations Gas Use', index=False)
# current_district_therm_loads.to_excel(writer, sheet_name='Current District Therm Op Loads', index=False)
# COP_wetbulb_reg_output.to_excel(writer, sheet_name='Reg. Data - Current District', index=False)
# current_District_Elec_Use.to_excel(writer, sheet_name='Current District Elec Use', index=False)
# current_District_Gas_Use.to_excel(writer, sheet_name='Current District Gas Use', index=False)
# current_Bldg_Therm_Op_Loads.to_excel(writer, sheet_name='Current Bldg Therm Op Loads', index=False)
# Current_Bldg_Equip_Elec_Use.to_excel(writer, sheet_name='Current Bldg Equip Elec Use', index=False)
# Current_Bldg_Equip_Gas_Use.to_excel(writer, sheet_name='Current Bldg Equip Gas Use', index=False)
# new_Bldg_Therm_Op_Loads.to_excel(writer, sheet_name='New Bldg Equip Therm Op Loads', index=False)
# COP_newBldgEquip_reg_output.to_excel(writer, sheet_name='Reg. Data - New Bldg Equip', index=False)
# new_bldg_equip_elec_Use.to_excel(writer, sheet_name='New Bldg Equip Elec Use', index=False)
# new_CUP_Therm_Loads.to_excel(writer, sheet_name='Therm Loads for New CUP Module', index=False)
# key_outputs_OG.to_excel(writer, sheet_name='New CUP Therm Op Loads', index=False)
# newCUp_reg_df.to_excel(writer, sheet_name='Reg. Data - New CUP Equip', index=False)
# new_CUP_equip_elec_Use.to_excel(writer, sheet_name='New CUP Equip Elec Use', index=False)
# bldg_total_elec_use.to_excel(writer, sheet_name='Building Total Electricity Use', index=False)
# bldg_total_gas_use.to_excel(writer, sheet_name='Building Total Gas Use', index=False)
# elec_billing_df.to_excel(writer, sheet_name='Net Elec Use by Billing Group', index=False)
# bldg_total_elec_grouped_df.to_excel(writer, sheet_name='Elec Cost by Billing Group', index=False)

In [34]:
# allBldgLoads_output.to_parquet('allBldgLoads_output.parquet', index=False)

# allBldgLoads_output.to_csv('Simulations Thermal Use.csv', index=False)
# allBldgElecUse_output.to_csv('Simulations Electricity Use.csv', index=False)
# allBldgGasUse_output.to_csv('Simulations Gas Use.csv', index=False)
# current_district_therm_loads.to_csv('Current District Therm Op Loads.csv', index=False)
# COP_wetbulb_reg_output.to_csv('Reg. Data - Current District.csv', index=False)
# current_District_Elec_Use.to_csv('Current District Elec Use.csv', index=False)
# current_District_Gas_Use.to_csv('Current District Gas Use.csv', index=False)
# current_Bldg_Therm_Op_Loads.to_csv('Current Bldg Therm Op Loads.csv', index=False)
# Current_Bldg_Equip_Elec_Use.to_csv('Current Bldg Equip Elec Use.csv', index=False)
# Current_Bldg_Equip_Gas_Use.to_csv('Current Bldg Equip Gas Use.csv', index=False)
# new_Bldg_Therm_Op_Loads.to_csv('New Bldg Equip Therm Op Loads.csv', index=False)
# COP_newBldgEquip_reg_output.to_csv('Reg. Data - New Bldg Equip.csv', index=False)
# new_bldg_equip_elec_Use.to_csv('New Bldg Equip Elec Use.csv', index=False)
# new_CUP_Therm_Loads.to_csv('Therm Loads for New CUP Module.csv', index=False)
# key_outputs_OG.to_csv('New CUP Therm Op Loads.csv', index=False)
# newCUp_reg_df.to_csv('Reg. Data - New CUP Equip.csv', index=False)
# new_CUP_equip_elec_Use.to_csv('New CUP Equip Elec Use.csv', index=False)
# bldg_total_elec_use.to_csv('Building Total Electricity Use.csv', index=False)
# bldg_total_gas_use.to_csv('Building Total Gas Use.csv', index=False)
# elec_billing_df.to_csv('Net Elec Use by Billing Group.csv', index=False)
# bldg_total_elec_grouped_df.to_csv('Elec Cost by Billing Group.csv', index=False)

In [35]:
# Define the folder path where you want to save the CSV files
output_folder = "csv_outputs"  # Change this to your desired folder name


# Define a dictionary with DataFrame names as keys and DataFrames as values
dataframes = {
    "Simulations Thermal Use": allBldgLoads_output,
    "Simulations Electricity Use": allBldgElecUse_output,
    "Simulations Gas Use": allBldgGasUse_output,
    "Current District Therm Op Loads": current_district_therm_loads,
    "Reg. Data - Current District": COP_wetbulb_reg_output,
    "Current District Elec Use": current_District_Elec_Use,
    "Current District Gas Use": current_District_Gas_Use,
    "Current Bldg Therm Op Loads": current_Bldg_Therm_Op_Loads,
    "Current Bldg Equip Elec Use": Current_Bldg_Equip_Elec_Use,
    "Current Bldg Equip Gas Use": Current_Bldg_Equip_Gas_Use,
    "New Bldg Equip Therm Op Loads": new_Bldg_Therm_Op_Loads,
    "Reg. Data - New Bldg Equip": COP_newBldgEquip_reg_output,
    "New Bldg Equip Elec Use": new_bldg_equip_elec_Use,
    "Therm Loads for New CUP Module": new_CUP_Therm_Loads,
    "New CUP Therm Op Loads": key_outputs_OG,
    "Reg. Data - New CUP Equip": newCUp_reg_df,
    "New CUP Equip Elec Use": new_CUP_equip_elec_Use,
    "Building Total Electricity Use": bldg_total_elec_use,
    "Building Total Gas Use": bldg_total_gas_use,
    "Net Elec Use by Billing Group": elec_billing_df,
    "Elec Cost by Billing Group": bldg_total_elec_grouped_df,
}

# Loop through the dictionary
for name, df in dataframes.items():
    # Calculate the number of chunks needed
    num_chunks = (
        len(df) // 1048576 + 1
    )  # Assuming each chunk will have at most 1,048,576 rows

    # Split the DataFrame into chunks
    chunks = np.array_split(df, num_chunks)

    # Save each chunk to a separate CSV file
    for i, chunk in enumerate(chunks, start=1):
        # Generate the file path for the CSV file
        file_path = os.path.join(output_folder, f"{name}_{i}.csv")

        # Save the chunk to the CSV file
        chunk.to_csv(file_path, index=False)

/Users/samueld/PycharmProjects/district-system/.venv/lib/python3.12/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


# If you are in the year of decarb (2045) and this is your first time here, go to file UCSB_Excel to Python. If you are not in the year of decarb(2025), then you are all done. Check out your results in the folder - Calculation_map_output.xlsx !


# If you are in the year of decarb (2045) and you already reran this file, then you are all done. Check out your results in the folder - Calculation_map_output.xlsx !
